# Unitary Transformations

## Unitary Transformations Among Occupied Molecular Orbitals

In Hartree-Fock theory, the molecular orbitals (MOs) are obtained by solving the **Roothaan-Hall equations**, resulting in a set of orthonormal orbitals ordered by energy. However, it's important to understand that the **set of occupied orbitals is not unique** — any **unitary transformation** among the occupied orbitals leads to an equally valid Hartree-Fock solution.

### What Does That Mean?

Let $\{\psi_1, \psi_2, \dots, \psi_n\}$ be the occupied molecular orbitals in an RHF calculation. If we apply a **unitary transformation** $U$ (i.e., a transformation that preserves orthonormality):

$$
\psi'_i = \sum_{j=1}^{n} U_{ji} \, \psi_j
$$

then the new set $\{\psi'_1, \dots, \psi'_n\}$ is still:
- Orthonormal,
- Occupied,
- And yields **the same total wavefunction** and **the same electron density**.

### Why Is This True?

In Hartree-Fock, the **Slater determinant** built from the occupied orbitals defines the total electronic wavefunction. A unitary transformation of the occupied orbitals corresponds to a rotation of the determinant rows, which changes only the determinant by a constant phase factor (i.e., it remains physically the same).

As a result:
- The **electron density** $\rho(\mathbf{r})$ remains unchanged.
- All **observable quantities** (energy, dipole moment, forces, etc.) remain unchanged.

### Consequences:

- The choice of canonical orbitals (ordered by orbital energies) is conventional, not unique.
- One can form **localized orbitals** (e.g., Foster-Boys or Pipek-Mezey localization) via unitary transformations of the occupied space — these are still valid HF solutions

In [1]:
from pyqint import MoleculeBuilder, HF

# load molecule
mol = MoleculeBuilder().from_name('CH4')
print(mol)

# perform (restricted) Hartree-Fock calculation using (small) p321g basis set
result = HF().rhf(mol, 'p321')

Cannot find module tqdm
Molecule: None
 C (0.000000,0.000000,0.000000)
 H (1.195756,1.195756,1.195756)
 H (-1.195756,-1.195756,1.195756)
 H (-1.195756,1.195756,-1.195756)
 H (1.195756,-1.195756,-1.195756)



In [2]:
from pytessel import PyTessel
import numpy as np
import trimesh
import pythreejs as p3
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from math import factorial
from scipy.special import assoc_laguerre
from scipy.special import lpmv
from scipy.special import sph_harm
from pyqint import PyQInt

integrator = PyQInt()

def generate_scalarfield(N, sz, res, orbc):
    x = np.linspace(-sz, sz, res)
    xx, yy, zz = np.meshgrid(x,x,x)
    grid = np.vstack([xx.flatten(), yy.flatten(), zz.flatten()]).reshape(3,-1).T

    field = integrator.plot_wavefunction(grid, orbc, result['cgfs']).reshape(res, res, res)
    return field

def generate_isosurfaces(N, sz, res, name, orbc):
    pytessel = PyTessel()
    field = generate_scalarfield(N, sz, res, orbc)
    unitcell = np.diag(np.ones(3) * 3 * 2)
    pytessel = PyTessel()
    isovalue = 0.03
    vertices, normals, indices = pytessel.marching_cubes(field.flatten(), field.shape, unitcell.flatten(), isovalue)
    pytessel.write_ply('%s_pos.ply' % name, vertices, normals, indices)
    vertices, normals, indices = pytessel.marching_cubes(field.flatten(), field.shape, unitcell.flatten(), -isovalue)
    pytessel.write_ply('%s_neg.ply' % name, vertices, normals, indices)

def visualize_mesh(name):
    mesh_pos = trimesh.load_mesh("%s_pos.ply" % name)
    mesh_neg = trimesh.load_mesh("%s_neg.ply" % name)

    geometry_pos = p3.BufferGeometry(
        attributes={
            "position": p3.BufferAttribute(mesh_pos.vertices.astype(np.float32), normalized=False),
            "index": p3.BufferAttribute(mesh_pos.faces.astype(np.uint32).ravel(), normalized=False)
        }
    )
    
    geometry_neg = p3.BufferGeometry(
        attributes={
            "position": p3.BufferAttribute(mesh_neg.vertices.astype(np.float32), normalized=False),
            "index": p3.BufferAttribute(mesh_neg.faces.astype(np.uint32).ravel(), normalized=False),
        }
    )

    solid_material_pos = p3.MeshStandardMaterial(
        color="#276419",
        side="DoubleSide",
        transparent=True,
        opacity=0.3
    )
    solid_material_neg = p3.MeshStandardMaterial(
        color="#8e0152",
        side="DoubleSide",
        transparent=True,
        opacity=0.3
    )

    mesh_object_pos = p3.Mesh(geometry=geometry_pos, material=solid_material_pos)
    mesh_object_neg = p3.Mesh(geometry=geometry_neg, material=solid_material_neg)
    mesh_object_pos.rotateY(-np.pi/2)
    mesh_object_neg.rotateY(-np.pi/2)

    # Wireframe Overlay
    wireframe_material = p3.LineBasicMaterial(color="black", linewidth=1.0)  # Thin black lines
    wireframe_pos = p3.LineSegments(
        p3.EdgesGeometry(geometry_pos),  # Extracts edges from geometry
        wireframe_material,
        transparent=True,
        opacity=0.3
    )
    wireframe_neg = p3.LineSegments(
        p3.EdgesGeometry(geometry_neg),  # Extracts edges from geometry
        wireframe_material,
        transparent=True,
        opacity=0.3
    )
    wireframe_pos.rotateY(-np.pi/2)
    wireframe_neg.rotateY(-np.pi/2)

    # Atom spheres: Carbon (black) at z = -1, Oxygen (red) at z = +1
    carbon_sphere = p3.Mesh(
        geometry=p3.SphereGeometry(radius=0.2, widthSegments=32, heightSegments=32),
        material=p3.MeshStandardMaterial(color="black")
    )
    carbon_sphere.position = [0, 0, 0]

    hydrogen_spheres = []
    for i in range(4):
        hydrogen_sphere = p3.Mesh(
            geometry=p3.SphereGeometry(radius=0.15, widthSegments=32, heightSegments=32),
            material=p3.MeshStandardMaterial(color="#999999")
        )
        hydrogen_sphere.position = tuple(result['nuclei'][i+1][0] / 1.8897259886)
        hydrogen_sphere.position = (hydrogen_sphere.position[0], -hydrogen_sphere.position[1], hydrogen_sphere.position[2])
        hydrogen_spheres.append(hydrogen_sphere)
    
    # Lighting setup
    ambient_light = p3.AmbientLight(color="white", intensity=4)
    
    # Create Scene
    scene = p3.Scene(children=[mesh_object_pos, 
                               mesh_object_neg, 
                               wireframe_pos, 
                               wireframe_neg, 
                               ambient_light,
                               carbon_sphere, 
                               hydrogen_spheres[0],
                               hydrogen_spheres[1],
                               hydrogen_spheres[2],
                               hydrogen_spheres[3],
                               p3.AxesHelper(size=5)
                              ])
    camera = p3.PerspectiveCamera(position=[5, 5, 5], fov=50)
    controller = p3.OrbitControls(controlling=camera)

    renderer = p3.Renderer(
        scene=scene, camera=camera, controls=[controller], 
        width=512, height=512, antialias=True
    )
    
    return renderer

def update_plot(N):
    name = 'MO_%03i' % N
    generate_isosurfaces(N-1, 5, 70, name, result['orbc'][:,N-1])
    renderer = visualize_mesh(name)
    display(renderer)
    
# Create an interactive slider to vary the number of basis functions
N_slider = widgets.IntSlider(
    min=1, max=9, step=1, value=0,
    description="Solution index"
)

# Link slider to update function and display interactive UI
widgets.interactive(update_plot, N=N_slider)

interactive(children=(IntSlider(value=1, description='Solution index', max=9, min=1), Output()), _dom_classes=…

## Canonical vs. Localized Molecular Orbitals in Methane (CH₄)

In this workflow, we explore the nature of molecular orbitals (MOs) in methane (CH₄) by comparing **canonical Hartree-Fock orbitals** with **Foster-Boys localized orbitals**. This comparison helps to bridge the gap between the mathematical formulation of quantum chemistry and the chemically intuitive picture of bonding.

### 1. Canonical Molecular Orbitals

First, we perform a Hartree-Fock (RHF) calculation on methane using the p321g basis set. The output includes the **canonical molecular orbitals**, which are:

- Eigenfunctions of the Fock operator.
- Delocalized over the entire molecule.
- Sorted by energy (lowest to highest).
- Mathematically optimal for the SCF procedure, but often **not chemically intuitive**.

We visualize these canonical orbitals using **isosurfaces** — 3D representations of constant wavefunction value (e.g., ±0.03), showing the shape, phase, and nodal structure of each orbital.

### 2. Foster-Boys Localization

Next, we apply the **Foster-Boys localization** procedure to the set of occupied orbitals. This involves a **unitary transformation** among the occupied MOs to maximize the spatial localization of each orbital. In particular, the Boys method minimizes the spread of each orbital by minimizing:

$$
\sum_i \left( \langle \psi_i | \mathbf{r}^2 | \psi_i \rangle - \langle \psi_i | \mathbf{r} | \psi_i \rangle^2 \right)
$$

The result is a set of **localized MOs** that:
- Still span the same occupied subspace (i.e., represent the same total wavefunction),
- Are orthonormal and valid HF orbitals,
- Tend to resemble **classical bonding pairs**, such as lone pairs or σ-bonds.

### 3. Visualization of Localized Orbitals

Finally, we visualize the **localized orbitals** using the same isosurface approach. In the case of methane, the result is typically four equivalent orbitals pointing toward the corners of a tetrahedron — corresponding closely to the classical picture of **four sp³ hybrid orbitals** forming **σ-bonds** between carbon and hydrogen.

This comparison highlights a powerful concept in quantum chemistry: although canonical orbitals are convenient for computations, **localized orbitals often align more closely with chemical intuition**, making them valuable for interpretation and teaching.

In [3]:
from pyqint import FosterBoys
resfb = FosterBoys(result).run()

def update_plot_fb(N):
    name = 'MO_%03i' % N
    generate_isosurfaces(N-1, 5, 70, name, resfb['orbc'][:,N-1])
    renderer = visualize_mesh(name)
    display(renderer)
    
# Create an interactive slider to vary the number of basis functions
N_slider = widgets.IntSlider(
    min=1, max=9, step=1, value=0,
    description="Solution index"
)

# Link slider to update function and display interactive UI
widgets.interactive(update_plot_fb, N=N_slider)

interactive(children=(IntSlider(value=1, description='Solution index', max=9, min=1), Output()), _dom_classes=…